# Sugeno Fuzzy System (First-Order)

**Linear Output Functions**

In [ ]:
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fuzzy_systems.inference import SugenoSystem

%matplotlib inline

### 1. Create Sugeno System

In [ ]:
# Create Sugeno system with linear output functions
fis = SugenoSystem()

# Add input variables
temp = fis.add_input('temp', (0, 40))
humidity = fis.add_input('humidity', (0, 100))

# Add terms to temperature
fis.add_term('temp', 'low', 'triangular', (0, 0, 20))
fis.add_term('temp', 'medium', 'triangular', (10, 20, 30))
fis.add_term('temp', 'high', 'triangular', (20, 40, 40))

# Add terms to humidity
fis.add_term('humidity', 'dry', 'triangular', (0, 0, 50))
fis.add_term('humidity', 'normal', 'triangular', (20, 50, 80))
fis.add_term('humidity', 'wet', 'triangular', (50, 100, 100))

# Visualize
fis.plot_variables(figsize=(14, 4))

### 2. First-Order Rules

In [ ]:
# First-order rules: output = f(temp, humidity)
fis.add_rule("IF temp IS low AND humidity IS dry THEN output = 0.5*temp + 0.1*humidity + 5")
fis.add_rule("IF temp IS medium AND humidity IS normal THEN output = 0.3*temp + 0.2*humidity + 10")
fis.add_rule("IF temp IS high AND humidity IS wet THEN output = 0.1*temp + 0.4*humidity + 15")

fis.print_rules()

### 3. Single Inference

In [ ]:
# Test specific input
result = fis.evaluate(temp=25, humidity=60)

print(f"Input: temp=25°C, humidity=60%")
print(f"Output: {result['output']:.2f}")

### 4. 3D Surface Plot

In [ ]:
# Generate surface over input space
temp_range = np.linspace(0, 40, 30)
hum_range = np.linspace(0, 100, 30)
output_surface = np.zeros((len(hum_range), len(temp_range)))

for i, h in enumerate(hum_range):
    for j, t in enumerate(temp_range):
        result = fis.evaluate(temp=t, humidity=h)
        output_surface[i, j] = result['output']

# 3D plot
from mpl_toolkits.mplot3d import Axes3D
T, H = np.meshgrid(temp_range, hum_range)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(T, H, output_surface, cmap='coolwarm', alpha=0.8)
ax.set_xlabel('Temperature (°C)')
ax.set_ylabel('Humidity (%)')
ax.set_zlabel('Output')
ax.set_title('First-Order Sugeno: Response Surface')
plt.colorbar(surf, shrink=0.5)
plt.show()

### 5. Contour Plot

In [ ]:
# 2D contour
plt.figure(figsize=(10, 8))
contour = plt.contourf(T, H, output_surface, levels=15, cmap='coolwarm')
plt.colorbar(contour, label='Output')
plt.xlabel('Temperature (°C)')
plt.ylabel('Humidity (%)')
plt.title('First-Order Sugeno: Contour Map')
plt.grid(True, alpha=0.3)
plt.show()

### 🎯 Exercise: Compare Zero vs First-Order

In [ ]:
# Create zero-order system for comparison
fis_zero = SugenoSystem()
fis_zero.add_input('temp', (0, 40))
fis_zero.add_input('humidity', (0, 100))
fis_zero.add_term('temp', 'low', 'triangular', (0, 0, 20))
fis_zero.add_term('temp', 'medium', 'triangular', (10, 20, 30))
fis_zero.add_term('temp', 'high', 'triangular', (20, 40, 40))
fis_zero.add_term('humidity', 'dry', 'triangular', (0, 0, 50))
fis_zero.add_term('humidity', 'normal', 'triangular', (20, 50, 80))
fis_zero.add_term('humidity', 'wet', 'triangular', (50, 100, 100))

# Constant outputs (zero-order)
fis_zero.add_rule("IF temp IS low AND humidity IS dry THEN output = 10")
fis_zero.add_rule("IF temp IS medium AND humidity IS normal THEN output = 20")
fis_zero.add_rule("IF temp IS high AND humidity IS wet THEN output = 30")

# Compare along temperature axis (humidity fixed at 50%)
temps = np.linspace(0, 40, 50)
out_zero = [fis_zero.evaluate(temp=t, humidity=50)['output'] for t in temps]
out_first = [fis.evaluate(temp=t, humidity=50)['output'] for t in temps]

plt.figure(figsize=(12, 5))
plt.plot(temps, out_zero, 'b-', linewidth=2, label='Zero-order (constant)')
plt.plot(temps, out_first, 'r-', linewidth=2, label='First-order (linear)')
plt.xlabel('Temperature (°C)')
plt.ylabel('Output')
plt.title('Zero-Order vs First-Order Sugeno (humidity=50%)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Notice: First-order has smooth linear transitions between rules")